# Entendimiento de datos

In [1]:
# Importaciones
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

## Utils

In [2]:
# Variables de configuración
SEED = 9603

MUESTRA = 1-0.125
# 0.3678028341415188 / 500K
# 0.1218641130361094 / 165K
# 0.2907504151915929

TEST_SIZE = 0.1

VALID_SIZE = 0.25

files_paths = [
  # "Monday-WorkingHours.csv",
  "Tuesday-WorkingHours.csv",
  "Wednesday-WorkingHours.csv",
  # "Thursday-WorkingHours.csv",
  "Friday-WorkingHours.csv",
]

In [3]:
# getFrequency(): Función para obtener la distribución de frecuencias de la columna label
def getFrequency(df : pd.DataFrame, caracteristica) :
  frecuencia = df[caracteristica].value_counts()
  porcentaje = df[caracteristica].value_counts(normalize=True) * 100

  tabla_frecuencia = pd.DataFrame({
    "Frecuencia": frecuencia,
    "Frecuencia(%)": porcentaje
  })
  
  tabla_frecuencia["Frecuencia(%)"].round(2)
  
  print(tabla_frecuencia)
  print(df.shape)

In [4]:
# generateCorrMatrix(): Función para generar matriz de correlación
def generateCorrMatrix(data, save_path="", title="Matriz de correlación", method="pearson", width=32, height=24, decimals=2, abs = False, save=True, aplicar_umbral=False, umbral=0.5) :
  # Generación de matriz de correlación
  matriz_correlacion = data.corr(method=method).round(decimals=decimals)

  if abs :
    matriz_correlacion = matriz_correlacion.abs()

  if aplicar_umbral : 
    matriz_correlacion = matriz_correlacion.where(np.abs(matriz_correlacion) >= umbral, 0)

  # Generación de mapa de calor para visualizar los datos
  plt.figure(figsize=(width, height))

  sns.heatmap(matriz_correlacion, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
  plt.title(title)
  plt.tight_layout()
  if save :
    plt.savefig(save_path)
  plt.show()
  return matriz_correlacion

## Carga de datos

In [5]:
files_paths_1 = [
  "Monday-WorkingHours.csv",
  "Tuesday-WorkingHours.csv",
  "Wednesday-WorkingHours.csv",
  "Thursday-WorkingHours.csv",
  "Friday-WorkingHours.csv",
]

df_1 = pd.concat(
  [pd.read_csv(f"DB/{file_path}") for file_path in files_paths_1]
)
getFrequency(df=df_1, caracteristica="Label")

                                      Frecuencia  Frecuencia(%)
Label                                                          
BENIGN                                   1657693      78.907176
PortScan                                  159151       7.575683
DoS Hulk                                  158469       7.543219
DDoS                                       95123       4.527912
DoS GoldenEye                               7567       0.360194
DoS slowloris                               4001       0.190450
FTP-Patator                                 3973       0.189117
DoS Slowhttptest - Attempted                3369       0.160366
SSH-Patator                                 2980       0.141850
DoS Slowhttptest                            1742       0.082920
DoS slowloris - Attempted                   1731       0.082397
Bot - Attempted                             1470       0.069973
Web Attack - Brute Force - Attempted        1214       0.057787
Bot                                     

In [6]:
# Carga de datos
df_monday = pd.read_csv(f"DB/Monday-WorkingHours.csv")
df_tuesday = pd.read_csv(f"DB/Tuesday-WorkingHours.csv")
df_wednesday = pd.read_csv(f"DB/Wednesday-WorkingHours.csv")
df_thursday = pd.read_csv(f"DB/Thursday-WorkingHours.csv")
df_friday = pd.read_csv(f"DB/Friday-WorkingHours.csv")

print("\nLunes")
getFrequency(df_monday, 'Label')
print("\nMartes")
getFrequency(df_tuesday, 'Label')
print("\nMiercoles")
getFrequency(df_wednesday, 'Label')
print("\nJueves")
getFrequency(df_thursday, 'Label')
print("\nViernes")
getFrequency(df_friday, 'Label')


Lunes
        Frecuencia  Frecuencia(%)
Label                            
BENIGN      371749          100.0
(371749, 84)

Martes
                         Frecuencia  Frecuencia(%)
Label                                             
BENIGN                       315031      97.834803
FTP-Patator                    3973       1.233839
SSH-Patator                    2980       0.925457
FTP-Patator - Attempted          11       0.003416
SSH-Patator - Attempted           8       0.002484
(322003, 84)

Miercoles
                              Frecuencia  Frecuencia(%)
Label                                                  
BENIGN                            319216      64.257145
DoS Hulk                          158469      31.899295
DoS GoldenEye                       7567       1.523213
DoS slowloris                       4001       0.805388
DoS Slowhttptest - Attempted        3369       0.678169
DoS Slowhttptest                    1742       0.350659
DoS slowloris - Attempted           1731 

In [7]:
# Integración de datos
df = pd.concat(
  [pd.read_csv(f"DB/{file_path}") for file_path in files_paths]
)
getFrequency(df=df, caracteristica="Label")

                              Frecuencia  Frecuencia(%)
Label                                                  
BENIGN                            925680      67.731180
PortScan                          159151      11.644937
DoS Hulk                          158469      11.595035
DDoS                               95123       6.960065
DoS GoldenEye                       7567       0.553671
DoS slowloris                       4001       0.292750
FTP-Patator                         3973       0.290701
DoS Slowhttptest - Attempted        3369       0.246507
SSH-Patator                         2980       0.218044
DoS Slowhttptest                    1742       0.127461
DoS slowloris - Attempted           1731       0.126656
Bot - Attempted                     1470       0.107559
Bot                                  738       0.053999
DoS Hulk - Attempted                 593       0.043389
DoS GoldenEye - Attempted             80       0.005854
FTP-Patator - Attempted               11       0

In [8]:
# Eliminación de etiquetas que no representan carga útil maliciosa y registros suficientes de un tipo de ciberataque para entrenar modelos
df = df[~df["Label"].isin([
  'DoS Slowhttptest - Attempted',
  'DoS slowloris - Attempted',
  'Bot - Attempted',
  'DoS Hulk - Attempted',
  'DoS GoldenEye - Attempted',
  'Infiltration - Attempted',
  'FTP-Patator - Attempted',
  'SSH-Patator - Attempted',
  'Infiltration',
  'Heartbleed',
])]
getFrequency(df=df, caracteristica="Label")

                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                925680      68.093546
PortScan              159151      11.707238
DoS Hulk              158469      11.657069
DDoS                   95123       6.997302
DoS GoldenEye           7567       0.556633
DoS slowloris           4001       0.294316
FTP-Patator             3973       0.292256
SSH-Patator             2980       0.219210
DoS Slowhttptest        1742       0.128143
Bot                      738       0.054288
(1359424, 84)


In [9]:
# Muestra para el proyecto
df_vector, df_temp = train_test_split(df, stratify=df["Label"], test_size=MUESTRA, random_state=SEED)
getFrequency(df=df_vector, caracteristica="Label")

                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                115710      68.093546
PortScan               19894      11.707311
DoS Hulk               19809      11.657290
DDoS                   11890       6.997081
DoS GoldenEye            946       0.556706
DoS slowloris            500       0.294242
FTP-Patator              497       0.292477
SSH-Patator              372       0.218916
DoS Slowhttptest         218       0.128290
Bot                       92       0.054141
(169928, 84)


In [10]:
# Conjunto de prueba
df_vector_train_valid, df_test = train_test_split(df_vector, stratify=df_vector["Label"], test_size=TEST_SIZE, random_state=SEED)
print("Conjunto de prueba")
getFrequency(df=df_test, caracteristica="Label")

Conjunto de prueba
                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                 11571      68.092744
PortScan                1989      11.704820
DoS Hulk                1981      11.657741
DDoS                    1189       6.996999
DoS GoldenEye             95       0.559054
DoS slowloris             50       0.294239
FTP-Patator               50       0.294239
SSH-Patator               37       0.217737
DoS Slowhttptest          22       0.129465
Bot                        9       0.052963
(16993, 84)


In [11]:
# Conjuntos de entrenamiento y validación OOB
df_train, df_valid = train_test_split(df_vector_train_valid, stratify=df_vector_train_valid["Label"], test_size=VALID_SIZE, random_state=9603)
print("Conjunto de entrenamiento")
getFrequency(df=df_train, caracteristica="Label")
print("\nConjunto de validación OOB")
getFrequency(df=df_valid, caracteristica="Label")

Conjunto de entrenamiento
                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                 78104      68.093565
PortScan               13429      11.707832
DoS Hulk               13371      11.657265
DDoS                    8026       6.997323
DoS GoldenEye            638       0.556229
DoS slowloris            338       0.294679
FTP-Patator              335       0.292064
SSH-Patator              251       0.218830
DoS Slowhttptest         147       0.128159
Bot                       62       0.054054
(114701, 84)

Conjunto de validación OOB
                  Frecuencia  Frecuencia(%)
Label                                      
BENIGN                 26035      68.093843
PortScan                4476      11.706858
DoS Hulk                4457      11.657164
DDoS                    2675       6.996391
DoS GoldenEye            213       0.557096
DoS slowloris            112       0.292933
FTP-Patator              112       0.292933
SSH-Patat

## Descripción de datos

In [12]:
# Características del dataset
X_columns = df_vector.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Label'], axis=1).columns
X_columns

Index(['Dst Port', 'Protocol', 'Flow Duration', 'Total Fwd Packet',
       'Total Bwd packets', 'Total Length of Fwd Packet',
       'Total Length of Bwd Packet', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
       'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count'

In [13]:
# Descripción de variables 
caracteristicas_numericas = [
  'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 
  'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 
  'Fwd Packet Length Max', 'Fwd Packet Length Min', 
  'Fwd Packet Length Mean', 'Fwd Packet Length Std', 
  'Bwd Packet Length Max', 'Bwd Packet Length Min', 
  'Bwd Packet Length Mean', 'Bwd Packet Length Std', 
  'Flow Bytes/s', 'Flow Packets/s',
  'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
  'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
  'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
  'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags',
  'Fwd Header Length',
  'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
  'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
  'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
  'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
  'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
  'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
  'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
  'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
  'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
  'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
  'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
  'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
  'Idle Min'
]

caracteristicas_var_max_inf = ['Flow Bytes/s', 'Flow Packets/s']

caracteristicas_no_utiles = ['Fwd URG Flags', 'Bwd URG Flags', 'URG Flag Count']

caracteristicas_nominales = ['Dst Port', 'Protocol']

caracteristica_objetivo = "Label"

## Exploración de datos

In [14]:
for i in caracteristicas_numericas : 
  print(f"""
  Característica: {i}
{df_vector[i].describe()}
  """)


  Característica: Flow Duration
count    1.699280e+05
mean     1.139761e+07
std      2.963552e+07
min     -1.000000e+00
25%      2.540000e+02
50%      6.984600e+04
75%      4.155948e+06
max      1.200000e+08
Name: Flow Duration, dtype: float64
  

  Característica: Total Fwd Packet
count    169928.000000
mean         13.797932
std        1022.690897
min           0.000000
25%           2.000000
50%           2.000000
75%           8.000000
max      203943.000000
Name: Total Fwd Packet, dtype: float64
  

  Característica: Total Bwd packets
count    169928.000000
mean         16.013523
std        1373.697882
min           0.000000
25%           2.000000
50%           2.000000
75%           6.000000
max      266796.000000
Name: Total Bwd packets, dtype: float64
  

  Característica: Total Length of Fwd Packet
count    169928.000000
mean        544.530689
std        5204.360417
min           0.000000
25%          45.000000
50%          80.000000
75%         355.000000
max      877610.000

c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)



  Característica: Bwd Packets/s
count     169928.000000
mean        5472.009503
std        26554.790293
min            0.000000
25%            1.629634
50%           33.077534
75%         7604.562738
max      1000000.000000
Name: Bwd Packets/s, dtype: float64
  

  Característica: Packet Length Min
count    169928.000000
mean         19.113936
std          25.151581
min           0.000000
25%           0.000000
50%           0.000000
75%          40.000000
max        1326.000000
Name: Packet Length Min, dtype: float64
  

  Característica: Packet Length Max
count    169928.000000
mean       1661.023457
std        2657.119623
min           0.000000
25%          63.000000
50%         139.000000
75%        2132.250000
max       24820.000000
Name: Packet Length Max, dtype: float64
  

  Característica: Packet Length Mean
count    169928.000000
mean        260.474094
std         333.830439
min           0.000000
25%          50.000000
50%          85.000000
75%         313.666667
max      

In [15]:
# las características flow_bytes/s y flow_packets/s tienen valores infinito 
for i in caracteristicas_var_max_inf : 
  print(f"""
  Característica: {i}
{df_vector[i].describe()}
  """)


  Característica: Flow Bytes/s
count    1.698970e+05
mean              inf
std               NaN
min      0.000000e+00
25%      1.307975e+02
50%      3.963179e+03
75%      7.081275e+04
max               inf
Name: Flow Bytes/s, dtype: float64
  

  Característica: Flow Packets/s
count    1.699280e+05
mean              inf
std               NaN
min     -2.000000e+06
25%      3.689204e+00
50%      7.610922e+01
75%      1.581028e+04
max               inf
Name: Flow Packets/s, dtype: float64
  


c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
c:\Users\User\anaconda3\envs\ids_thesis\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [16]:
# Característica Bwd URG Flags no tiene información útil para el modelo
for i in caracteristicas_no_utiles : 
  print(f"""
  Característica: {i}
{df_vector[i].describe()}
  """)



  Característica: Fwd URG Flags
count    169928.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: Fwd URG Flags, dtype: float64
  

  Característica: Bwd URG Flags
count    169928.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: Bwd URG Flags, dtype: float64
  

  Característica: URG Flag Count
count    169928.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: URG Flag Count, dtype: float64
  


In [17]:
# Visualización de valores nulos
null_chars_list = df_vector.isna().sum()
null_chars_list = null_chars_list[null_chars_list > 0]
null_chars_list

Flow Bytes/s     31
Flow IAT Mean    13
Flow IAT Std     13
Flow IAT Max     13
Flow IAT Min     13
dtype: int64

In [18]:
# Visualización de valores infinitos
inf_chars_list = np.isinf(df_vector[caracteristicas_numericas]).sum()
inf_chars_list = inf_chars_list[inf_chars_list > 0]
inf_chars_list

Flow Bytes/s      13
Flow Packets/s    44
dtype: int64

### Valores nulos

In [20]:
df_vector[["Fwd Packets/s", "Bwd Packets/s", "Flow Packets/s"]]

,Fwd Packets/s,Bwd Packets/s,Flow Packets/s
74880,63.447751,63.447751,126.895502
215476,0.287489,0.287489,0.574978
225912,2.242097,1.245610,3.487707
148112,35.593649,21.356189,56.949838
193627,138.858032,111.086425,249.944457
...,...,...,...
322074,2156.500308,2156.500308,4313.000616
304799,129.979853,64.989927,194.969780
77938,46.680982,46.680982,93.361964
156874,1.361965,1.021474,2.383438


In [21]:
df_comparacion = pd.concat([
  (df_vector["Fwd Packets/s"] + df_vector["Bwd Packets/s"]).to_frame(),
  df_vector["Flow Packets/s"]
], axis=1)

df_comparacion.columns = ["Flow Packets/s.1", "Flow Packets/s"]

df_comparacion[~np.isclose(df_comparacion["Flow Packets/s.1"], df_comparacion["Flow Packets/s"])]["Flow Packets/s.1"].unique()

array([0.])

In [22]:
df_vector = df_vector.drop(['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp'] + caracteristicas_no_utiles, axis=1)

## Giardado


In [23]:
# Transformación de nombres de columnas
df_vector.columns = df_vector.columns.str.strip().str.lower().str.replace(" - ", "_").str.replace(" ", "_")
print(df_vector.columns)
print(f"Total de características: {len(df_vector.columns)}")

Index(['dst_port', 'protocol', 'flow_duration', 'total_fwd_packet',
       'total_bwd_packets', 'total_length_of_fwd_packet',
       'total_length_of_bwd_packet', 'fwd_packet_length_max',
       'fwd_packet_length_min', 'fwd_packet_length_mean',
       'fwd_packet_length_std', 'bwd_packet_length_max',
       'bwd_packet_length_min', 'bwd_packet_length_mean',
       'bwd_packet_length_std', 'flow_bytes/s', 'flow_packets/s',
       'flow_iat_mean', 'flow_iat_std', 'flow_iat_max', 'flow_iat_min',
       'fwd_iat_total', 'fwd_iat_mean', 'fwd_iat_std', 'fwd_iat_max',
       'fwd_iat_min', 'bwd_iat_total', 'bwd_iat_mean', 'bwd_iat_std',
       'bwd_iat_max', 'bwd_iat_min', 'fwd_psh_flags', 'bwd_psh_flags',
       'fwd_header_length', 'bwd_header_length', 'fwd_packets/s',
       'bwd_packets/s', 'packet_length_min', 'packet_length_max',
       'packet_length_mean', 'packet_length_std', 'packet_length_variance',
       'fin_flag_count', 'syn_flag_count', 'rst_flag_count', 'psh_flag_count',
   

In [24]:
# Transformación de nombres de etiquetas de la columna label
df_vector["label"] = df_vector["label"].str.upper().str.replace("� ", "").str.replace(" - ", "_").str.replace(" ", "_").str.replace("-", "_")
print(df_vector["label"].unique())
print(f"Total de etiquetas: {len(df_vector["label"].unique())}")

['BENIGN' 'DDOS' 'DOS_HULK' 'PORTSCAN' 'DOS_GOLDENEYE' 'FTP_PATATOR'
 'SSH_PATATOR' 'DOS_SLOWHTTPTEST' 'DOS_SLOWLORIS' 'BOT']
Total de etiquetas: 10


In [25]:
df_vector.to_csv("output/dataset.csv", index=False)
df_vector.to_csv("../3_preparacion_de_datos/DB/dataset.csv", index=False)
df_vector.to_csv("../implementation/DB/dataset.csv", index=False)
print("Guardado")

Guardado
